# Introduction

The final calendar code will be contained in 2 files. One is labelled JivaCalendar_GCRS and the other JivaCalendar_Ecliptic. This is because there are two ways of doing calculation, and I'm not sure which one the Indian system uses. I think the Ecliptic one is the one we need but I'm keeping both intact for now. 

# Dependencies and Utils

In [1]:
from astropy import units as u
from astropy.time import Time
from astropy.coordinates import solar_system_ephemeris, EarthLocation
from astropy.coordinates import get_body_barycentric, get_body, get_moon, get_sun
from astropy.coordinates import SkyCoord, GCRS, ICRS, Angle, GeocentricTrueEcliptic, GeocentricMeanEcliptic
import numpy as np
from scipy.optimize import fsolve
from datetime import date, timedelta, datetime, time

!pip install jplephem
from astropy.coordinates import solar_system_ephemeris
solar_system_ephemeris.set('jpl')

!pip install mathjspy
from mathjspy import MathJS
mjs = MathJS()

In [2]:
## ----------UTILS------------
def datetime_to_astropy(date_):
  # date_ is a datetime.date object. We want to convert it to astropy.time.Time
    return Time(date_.strftime('%Y-%m-%d %H:%M:%S'))

def astropy_to_date(t): # Converts only to date. Truncates
    date_,time = t.value.split()
    year,month,day = [mjs.eval(s) for s in date_.split('-')]
    return date(year,month,day)

def astropy_to_datetime(t):
    date_,time = t.value.split()
    year,month,day = [mjs.eval(s) for s in date_.split('-')]
    hr,min,sec = [mjs.eval(s) for s in time.split(':')]
    return datetime(year,month,day,hr,min,int(sec))

Rasi_list = ["Mesa","Rsabha","Mithuna","Karkataka","Simha","Kanya","Tula","Vrscika","Dhanus","Makara","Kumbha","Meena"]

Maasa_list = ["Vaisakha","Jyestha","Asadha","Sravana","Bhadrapada","Asvina","Kartika","Mrgashirsha","Pausa","Magha","Phalguna","Caitra"]

Naksatra_list = ['Ashwini', 'Bharani', 'Krittika', 'Rohini', 'Mrighasira', 'Ardra', 'Punarvasu', 'Pushya', 'Ashlesha', 'Magha', 
                 'Purva Phalguni', 'Uttara Phalguni', 'Hasta', 'Chitra', 'Swati', 'Vishaka', 'Anuradha', 'Jyestha', 'Moola', 
                 'Purvashada', 'Uttarashada', 'Sharavan', 'Dhanishta', 'Shatabisha', 'Purvabhadra', 'Uttarabhadra', 'Revati']

maasa_gaps = [28, 33, 34, 35] 
# NOTE: I don't actually use this at all. My method is more fundamental, so is more reliable. Read the documentation for more info
# The gaps in adhika maasa, in months. 
# I think they're actually lunar months though, since the mean of the list is exactly 32.5. Predictions also seem to be correct with this assumption.
# Source: "https://sriramgurujala.com/what-is-adhik-maasam-or-adhik-maas-how-do-we-calculate-it/"

lunar_month = 29.5306 #length of lunar month in days

coord_Revati_ICRS = SkyCoord(ra='01h13m45.17477s', dec='+7d34m31.2745s', frame='icrs', equinox=Time(2000, format='jyear')) 
coord_Revati_Ec = coord_Revati_ICRS.transform_to(GeocentricTrueEcliptic()) # Geocentric True Ecliptic coordinates of Revati (Zeta Piscium A)
ayanamsa_revati = coord_Revati_Ec.lon
#coord_Revati_ICRS = SkyCoord(ra='01h13m45.17477s', dec='+7d34m31.2745s', distance=170*u.lyr, frame='icrs', equinox=Time(2000, format='jyear')) 
# The coordinate conversions still work even if we don't specify any distance. 
# Maybe they treat the distance as infinity? Because the resulting GCRS was barely different from ICRS.

coord_Spica_ICRS = SkyCoord(ra='13h25m11.579s', dec='−11d09m40.75s', frame='icrs', equinox=Time(2000, format='jyear')) 
coord_Spica_Ec = coord_Spica_ICRS.transform_to(GeocentricTrueEcliptic())
ayanamsa_citrapaksa = coord_Spica_Ec.lon - Angle('180d')

# Getting Solar and Lunar Longitudes etc

In [3]:
def get_sun_moon_Ec(t = Time("J2000")): # Ec=Ecliptic
    m = get_body('moon',t)
    m_inf = SkyCoord(ra=m.ra, dec=m.dec, frame='gcrs')
    m_ec = m_inf.transform_to(GeocentricTrueEcliptic())

    s = get_body('sun',t) 
    s_inf = SkyCoord(ra=s.ra, dec=s.dec, frame='gcrs')
    s_ec = s_inf.transform_to(GeocentricTrueEcliptic())
    return m_ec,s_ec

def get_angle_tithi_Ec(t= Time("J2000")):
    m,s = get_sun_moon_Ec(t=t)

    m_ra = m.lon.degree
    m_dec = m.lat.degree

    s_ra = s.lon.degree
    s_dec = s.lat.degree

    ms_angle = m_ra - s_ra # moon-sun angle. 
    ms_angle = ms_angle%360 
    tithi = (ms_angle)/12

    return ms_angle,tithi

# Finding New Moon and Solving for time

In [4]:
def find_new_moon_Ec(t_approx):
    # Find the exact date (+/-1 day) of new moon, given an approximate date. 
    # Input either in astropy.time.Time or datetime.date
    date_ = astropy_to_date(t_approx) # Extracting only the date. No time
    t = datetime_to_astropy(date_)
    t_af = Time(datetime_to_astropy(date_+timedelta(days=1)))  # one day later
    t_af2 = Time(datetime_to_astropy(date_+timedelta(days=2)))  # 2 days later
    t_bef = Time(datetime_to_astropy(date_-timedelta(days=1))) # one day before
    ang,tit = get_angle_tithi_Ec(t) 
    ang_bef, tit_bef = get_angle_tithi_Ec(t_bef)
    ang_af, tit_af = get_angle_tithi_Ec(t_af)
    ang_af2, tit_af2 = get_angle_tithi_Ec(t_af2)

    if ang_bef>330 and ang<30:
      correct_date = t_bef
    elif ang>330 and ang_af<30:
      correct_date = t
    elif ang_af>330 and ang_af2<30:
      correct_date = t_af
    else:
      return 0
    return correct_date

def find_new_moon_date_before_Ec(t=Time("J2000")):
    # Given a date, find the date the month starts by calculating the nearest earler new_moon
    ang,_ = get_angle_tithi_Ec(t)
    approx_delta = timedelta(days=ang/360*lunar_month)
    approx_date = datetime_to_astropy(astropy_to_date(t)-approx_delta)
    exact_date = find_new_moon_Ec(approx_date)
    if exact_date==0:
        new_approx_date = datetime_to_astropy(astropy_to_date(approx_date) + timedelta(days=-3))
        exact_date = find_new_moon_Ec(new_approx_date)
    if exact_date==0:
        new_approx_date = datetime_to_astropy(astropy_to_date(approx_date) + timedelta(days=+3))
        exact_date = find_new_moon_Ec(new_approx_date)
    return exact_date

def find_new_moon_time_Ec(t=Time("J2000"),accuracy=0.1):
    # Using the above two functions,  this one finds the exact new moon date and time. 
    # accuracy is the maximum angle difference in degrees from 360deg 
    date_ = find_new_moon_date_before_Ec(t=t)
    ang,_ = get_angle_tithi_Ec(date_)
    ang = 360 - ang
    approx_time = datetime.combine(astropy_to_date(date_),time(0,0,0))
    iter = 0
    while abs(ang)>accuracy:
        iter += 1
        if iter>100:
            raise Exception("Exceeded 100 iterations inside find_new_moon_time")
        approx_delta = timedelta(seconds=ang/360*lunar_month*24*60*60)
        approx_time += approx_delta 
        ang,_ = get_angle_tithi_Ec(t=datetime_to_astropy(approx_time))
        
        if ang<100:
            ang = -ang
        else:
            ang = 360 - ang
        # This if else statement below is so that if moon longitude<sun, then ang>0, else ang>0. 
        # By default if moon is 2 degrees behind sun, then ang=358. We want that to be 2degrees
        # if moon-sun=2degrees then ang=-2
        # RETIRING THIS because if moon lon = 359deg, and sun=0.5deg, this causes problems. Eg for the new_moon around 21st Mar, 2099. 
        #m,s = get_sun_moon_Ec(datetime_to_astropy(approx_time))
        #if m.lon<s.lon:   
        #  ang = 360 - ang
        #else:
        #  ang = -ang

    return datetime_to_astropy(approx_time)



In [5]:
## --------------------NOTE:: Need to build in some method below to avoid error close to 360 degree.
def solve_moon_time_Ec(lon,t,accuracy=0.1):
    # in the month of t, solve for the time at which sun-moon=lon
    nm_date = find_new_moon_time_Ec(t=t) # new_moon date
    nm_date = astropy_to_datetime(nm_date)
    approx_time = nm_date + timedelta(hours=lon/360*lunar_month*24)
    ang,_ = get_angle_tithi_Ec(datetime_to_astropy(approx_time))
    del_ang = lon - ang
    iter = 0
    while abs(del_ang)>accuracy:
        iter += 1
        if iter>100:
            raise Exception("Exceeded 100 iterations inside find_new_moon_time")
        approx_delta = timedelta(seconds=del_ang/360*lunar_month*24*60*60)
        approx_time += approx_delta 
        ang,_ = get_angle_tithi_Ec(t=datetime_to_astropy(approx_time))
        del_ang = lon-ang
    return datetime_to_astropy(approx_time)
        # This if else statement below is so that if moon longitude<sun, then ang>0, else ang>0. 
        # By default if moon is 2 degrees behind sun, then ang=358. We want that to be 2degrees
        # if moon-sun=2degrees then ang=-2


### The func below is to do the same job but with fsolve. 
### It does't work because fsolve needs to pass arrays as the input, and datetime etc dont allow that.
def find_new_moon_time_fsolve_Ec(t=Time("J2000"),accuracy=1):
    # Using the above two functions, find the exact new moon date. 
    # Then use this one to find the time.
    # accuracy is the maximum angle difference in degrees from 360deg 
    ang,tit = get_angle_tithi_Ec(t)
    approx_date = astropy_to_datetime(t) - timedelta(days=ang/360*lunar_month)
    print(approx_date)
    def ang_solver(t_):
      del_ = timedelta(minutes=t_)
      ang_,_ = get_angle_tithi_Ec(approx_date+del_)
      print(ang_)
      return ang_a
    
    t_solution = fsolve(ang_solver, 0)
    datetime_solution =  approx_date + timedelta(t_solution)
    print(t_solution)
    print(datetime_solution)
    #return datetime_to_astropy(approx_time)

# Getting Naksatra and Rasi Longitudes, and Finding Naksatra and Rasi for a given Longitude

In [6]:
def get_ayanamsa(ayanamsa):
    # Ayanamsa lookup table
    if type(ayanamsa) in [float,int]: return Angle(f"{ayanamsa}d")
    elif type(ayanamsa)==Angle: return ayanamsa
    elif type(ayanamsa)==str: 
        ayanamsa = ayanamsa.lower()
        if ayanamsa in ['lahiri','chitra','citra','spica','citrapaksa','chitrapaksa','citrapaksha','chitrapaksha']:
            return ayanamsa_citrapaksa
        if ayanamsa in ['revati','zeta piscium','piscium']:
            return ayanamsa_revati
        else:
            raise Exception("ayanamsa can be 'citrapaksa' or 'revati'")
    else:
        raise Exception("ayanamsa must be float, int, string or astropy.coordinates.angles.Angle")

def naksatra_lon_Ec(ayanamsa='citrapaksa'):
    # List of Naksatra longitudes (in the ecliptic, so equally spaced). With J2000 equinox as coordinate axis. 
    # Ayanamsa is the starting point of the Naksatras
    ayanamsa = get_ayanamsa(ayanamsa)
    diff = Angle('13d20m')
    naksatra_lon_list = [(ayanamsa + n*diff)%Angle("360d") for n in range(27)]
    return naksatra_lon_list

def rasi_lon_Ec(ayanamsa='citrapaksa'):
    # List of Rāśi longitudes (in the ecliptic, so equally spaced). With J2000 equinox as coordinate axis, and geocentric origin.
    # Ayanamsa is the starting point of the Naksatras
    ayanamsa = get_ayanamsa(ayanamsa)
    diff = Angle('30d')
    rasi_lon = [(ayanamsa + n*diff)%Angle("360d") for n in range(12)]
    return rasi_lon

# Note that the below functions (to find Naksatra for a given longitude) don't use the above functions to generate a list of 
# Naksatra longitudes. Just because there is no need to, so just to save time and compute.
def find_naksatra_Ec(lon,ayanamsa='citrapaksa'):
    if type(lon) in [int,float]: lon = Angle(f"{lon}d")
    ayanamsa = get_ayanamsa(ayanamsa)
    lon = (lon-ayanamsa)%Angle("360d")
    num = int(lon/Angle("13d20m"))
    nak = Naksatra_list[num]
    return num, nak

def find_rasi_Ec(lon,ayanamsa='citrapaksa'):
    if type(lon) in [int,float]: lon = Angle(f"{lon}d")
    ayanamsa = get_ayanamsa(ayanamsa)
    lon = (lon-ayanamsa)%Angle("360d")
    num = int(lon/Angle("30d"))
    rasi = Rasi_list[num]
    return num, rasi

# High Level Pancanga Functions

In [15]:
def find_tithi_start_end(t=Time("J2000"),accuracy=0.01):
    datetime_ = astropy_to_datetime(t)
    ang,tit = get_angle_tithi_Ec(t)
    s_lon = ang - ang%12 # start angle value for the tithi
    f_lon = s_lon + 12 # finish angle value for the tithi
    if s_lon==0:
        s_time = find_new_moon_time_Ec(t=t,accuracy=accuracy)
    else:
        s_time = solve_moon_time_Ec(s_lon,t,accuracy=accuracy)
    f_time = solve_moon_time_Ec(f_lon,t,accuracy=accuracy)
    tit = int(s_lon/12)+1
    info = f"tithi containing the time {t}"
    data = {"tithi":tit,"tithi start time":s_time,"tithi end time":f_time}
    return {"info":info,"data":data}

def find_masa_start_end_date(t=Time("J2000"),accuracy=0.01,ayanamsa='citrapaksa'):
    datetime_s = astropy_to_datetime(t)
    time_s = find_new_moon_time_Ec(t,accuracy=accuracy)
    datetime_e = astropy_to_datetime(time_s) + timedelta(days=(lunar_month+2)) 
    time_e = find_new_moon_time_Ec(datetime_to_astropy(datetime_e),accuracy=accuracy)
    m,s = get_sun_moon_Ec(time_s)
    num,_ = find_rasi_Ec(s.lon,ayanamsa=ayanamsa)
    masa = Maasa_list[num]
    sun_naksatra = find_naksatra_Ec(s.lon,ayanamsa=ayanamsa)[1]
    # sun_naksatra is only correct at the start of the month
    info = f"the māsa containing the time {t}"
    data = {"masa start time":time_s,"masa end time":time_e,"masa": masa,"sun naksatra at month beginning": sun_naksatra}
    return {"info":info,"data":data}

In [24]:
def get_pancanga_instant_Ec(t=Time("2021-5-31 10:00:00"),accuracy=0.01,ayanamsa='citrapaksa'):
    dict_ = find_masa_start_end_date(t=t,accuracy=accuracy,ayanamsa=ayanamsa)
    masa = dict_["data"]["masa"]
    dict_ = find_tithi_start_end(t=t,accuracy=accuracy)["data"]
    tithi, s_time, f_time = dict_["tithi"],dict_["tithi start time"],dict_["tithi end time"]
    m,s = get_sun_moon_Ec(t)
    _,m_naksatra = find_naksatra_Ec(lon=m.lon,ayanamsa=ayanamsa)
    _,s_naksatra = find_naksatra_Ec(lon=s.lon,ayanamsa=ayanamsa)
    info = f"pancanga for the time {t}"
    data = {"tithi":tithi,"masa":masa,"moon naksatra":m_naksatra,"sun naksatra":s_naksatra}
    return {"info":info,"data":data}

# Combine the function below with the above function
def get_pancanga_day_Ec():
    return tithi, masa, m_naksatra, s_naksatra, sunrise, sunset, moonrise, moonset

In [17]:
def generate_pancanga_lunar_month(t=Time("2021-5-31 10:00:00"),accuracy=0.01,ayanamsa='citrapaksa',verbose=True):
    nm_time,_,masa,s_nm_nak = find_masa_start_end_date(t=t,accuracy=accuracy,ayanamsa=ayanamsa)
    data = [(1,nm_time)]
    for i in range(1,30):
      if verbose and i%5==0: 
        print("running tithi:",i+1," "*20)
      lon = 12*i
      time_ = solve_moon_time_Ec(lon,t,accuracy=accuracy)
      data += [(i+1,time_)]
    info = {"This file contains tithis and their start times, for the masa containing the time":t,"sun naksatra at month beginning:":s_nm_nak,"Maasa:":masa}
    return info,data

In [25]:
get_pancanga_instant_Ec(t=Time("2021-5-31 10:00:00"),accuracy=0.01,ayanamsa='citra')

{'data': {'masa': 'Vaisakha',
  'moon naksatra': 'Sharavan',
  'sun naksatra': 'Rohini',
  'tithi': 21},
 'info': 'pancanga for the time 2021-05-31 10:00:00.000'}

# Below I do some unit tests

In [ ]:
# generating the names and start_time for all months from Jan 1993 to Nov 2028
t_initial = datetime(1990,1,15,0,0,0)
t = t_initial
months = []
all_dates = []
for i in range(40*12):
  if i%12==0: print(i%12+1)
  ts,te,mon = find_masa_start_end_date(datetime_to_astropy(t),accuracy=0.01,cor=4.2)
  t = te + timedelta(days=3)
  months += [mon]
  all_dates += [ts] 

In [ ]:
adhik = []
times = []
for i,mon in enumerate(months[0:-1]):
  nm = months[i+1] 
  if mon==nm: 
    adhik += [mon]
    times += [all_dates[i]]

In [ ]:
print("-------ADHIKA MASA LIST from Jan 1990 to Nov 2028---------")
print("------------start time of the month in UTC---------------")
for i,mon in enumerate(adhik):
  print(adhik[i],times[i])

-------ADHIKA MASA LIST from Jan 1990 to Nov 2028---------
------------start time of the month in UTC---------------
Vaisakha 1991-04-14 19:38:22.000
Bhadrapada 1993-08-17 19:27:48.000
Asadha 1996-06-16 01:35:07.000
Jyestha 1999-05-15 12:04:16.000
Asvina 2001-09-17 10:26:51.000
Sravana 2004-07-17 11:23:15.000
Jyestha 2007-05-16 19:26:33.000
Vaisakha 2010-04-14 12:27:42.000
Bhadrapada 2012-08-17 15:54:01.000
Asadha 2015-06-16 14:04:50.000
Jyestha 2018-05-15 11:48:38.000
Asvina 2020-09-17 10:59:51.000
Sravana 2023-07-17 18:31:01.000
Jyestha 2026-05-16 20:00:05.000


In [ ]:
# Calculating the entire months containing the dates below
big_data = []
dates = ["1993-7-20 13:00", "2001-4-20 13:00", "2015-9-20 13:00", "2021-1-20 13:00", "2026-3-20 13:00", "2034-6-20 13:00"]
for i,date_ in enumerate(dates):
  print("RUNNING",i+1)
  info, data = generate_pancanga_lunar_month(Time(date_),accuracy=0.01,verbose=True)
  times_a = [p[1] for p in data]
  print()
  print(f"The month of {date_}")
  print(info)
  print("tithi     start time")
  for i in range(1,31):
    print(i,times_a[i-1])
  print()


RUNNING 1
running tithi: 6                     
running tithi: 11                     
running tithi: 16                     
running tithi: 21                     
running tithi: 26                     

The month of 1993-7-20 13:00
{'sun naksatra at month beginning:': 'Punarvasu', 'Maasa:': 'Sravana'}
tithi     start time
1 1993-07-19 11:24:29.000
2 1993-07-20 08:53:27.000
3 1993-07-21 06:04:51.000
4 1993-07-22 03:07:17.000
5 1993-07-23 00:09:27.000
6 1993-07-23 21:19:08.000
7 1993-07-24 18:42:47.000
8 1993-07-25 16:25:18.000
9 1993-07-26 14:29:54.000
10 1993-07-27 12:58:20.000
11 1993-07-28 11:51:35.000
12 1993-07-29 11:09:03.000
13 1993-07-30 10:48:40.000
14 1993-07-31 10:53:05.000
15 1993-08-01 11:19:05.000
16 1993-08-02 12:09:50.000
17 1993-08-03 13:24:47.000
18 1993-08-04 15:03:34.000
19 1993-08-05 17:04:03.000
20 1993-08-06 19:21:32.000
21 1993-08-07 21:48:10.000
22 1993-08-09 00:13:10.000
23 1993-08-10 02:23:59.000
24 1993-08-11 04:06:11.000
25 1993-08-12 05:10:03.000
26 1993-

/usr/local/lib/python3.7/dist-packages/erfa/core.py:155: ErfaWarning: ERFA function "dtf2d" yielded 1 of "dubious year (Note 6)"
  ErfaWarning)
/usr/local/lib/python3.7/dist-packages/erfa/core.py:155: ErfaWarning: ERFA function "utctai" yielded 1 of "dubious year (Note 3)"
  ErfaWarning)
/usr/local/lib/python3.7/dist-packages/erfa/core.py:155: ErfaWarning: ERFA function "taiutc" yielded 1 of "dubious year (Note 4)"
  ErfaWarning)
/usr/local/lib/python3.7/dist-packages/erfa/core.py:155: ErfaWarning: ERFA function "d2dtf" yielded 1 of "dubious year (Note 5)"
  ErfaWarning)


running tithi: 6                     
running tithi: 11                     
running tithi: 16                     
running tithi: 21                     
running tithi: 26                     

The month of 2034-6-20 13:00
{'sun naksatra at month beginning:': 'Mrighasira', 'Maasa:': 'Asadha'}
tithi     start time
1 2034-06-16 10:25:42.000
2 2034-06-17 07:15:35.000
3 2034-06-18 04:31:46.000
4 2034-06-19 02:24:01.000
5 2034-06-20 01:00:31.000
6 2034-06-21 00:25:54.000
7 2034-06-22 00:43:23.000
8 2034-06-23 01:47:16.000
9 2034-06-24 03:32:14.000
10 2034-06-25 05:46:21.000
11 2034-06-26 08:12:29.000
12 2034-06-27 10:42:13.000
13 2034-06-28 13:00:19.000
14 2034-06-29 14:59:27.000
15 2034-06-30 16:34:47.000
16 2034-07-01 17:44:27.000
17 2034-07-02 18:28:38.000
18 2034-07-03 18:48:06.000
19 2034-07-04 18:45:31.000
20 2034-07-05 18:20:30.000
21 2034-07-06 17:33:13.000
22 2034-07-07 16:24:43.000
23 2034-07-08 14:53:24.000
24 2034-07-09 12:59:46.000
25 2034-07-10 10:45:03.000
26 2034-07-11 08:1

# Finding the value of precession correction 

Best value from Adika Maasa from 1910 to 2021: cor=**[3.94, 4.08]** or asvina_start=**[23.82, 23.96]** (J2000 ecliptic coordinates)

Update: It is actually Lahiri ayanamsa, which is **23.8564** at J2000, i.e. cor = **3.977**

The Lahiri Ayanamsa is actually based on Citrapaksa, i.e. 180° opposite Citra (Spica). Based on the actual coordinates of Spica (ra=13h25m11.579s, dec=−11°09′40.75″ in J2000 equinox ICRS coordinates), we get ayanamsa = **23.836**

Most reliable value so far: ***ayanamsa = 23.836***


i.e. Asvini naksatra starts at Zeta Piscium + cor.

The lahiri calculation uses Spica (Chitra) at 0 Libra (source:http://www.saravali.de/articles/aya_basics.html . Please verify.)

### Using last many years of Adhika maasa to constrain the correction
Note that the correct thing to do below is to not account for Earth's precession. This is because every GCRS and Ecliptic coordinate value is with respect to the J2000 equinox. Thus, the Earth can precess all it wants, but since we're in the J2000 frame, we only have a fixed deviation of the Naksatras.

In [ ]:
def cor_range(date_,accuracy=0.01,precession=False,start=3.5,end=4.3,step=0.1):
    st,et,_,_ = find_masa_start_end_date(date_,accuracy=accuracy,cor=0.0) # st=start_time, et=end_time
    _,ss = get_sun_moon_Ec(st) # ss,es = start and end sun positions respectively
    _,es = get_sun_moon_Ec(et)
    date_ = astropy_to_datetime(date_)
    delta_ = (date_ - datetime(2000,1,1,0,0,0)).days/(26000*365)*360
    if not precession: delta_=0
    print("delta:",delta_)

    cor_cor = []
    for cor in np.arange(start,end,step):
        sr,er = find_rasi_Ec(ss.lon,cor=cor-delta_),find_rasi_Ec(es.lon,cor=cor-delta_) # sn,en = start and end rasi respectively.
        #sr = find_rasi_Ec(ss.lon,cor=cor)
        #er = find_rasi_Ec(es.lon,cor=cor)
        if sr==er:
          cor_cor += [cor]
    if cor_cor == []: return -1,-1
    return np.min(cor_cor),np.max(cor_cor)

In [ ]:
# adhik maasa-s occur for the dates below
dates_ = [date(1993,8,19),date(1996,6,17),date(1999,5,17),date(2001,9,18),date(2004,7,19),date(2007,5,18),date(2010,4,16),date(2012,8,19),date(2015,6,18)]

mins = []
maxs = []
for i,date_ in enumerate(dates_):
    date_ = datetime_to_astropy(date_)
    c_min,c_max = cor_range(date_)
    mins += [c_min]
    maxs += [c_max]
print(mins)
print(maxs)
print("range:",np.max(mins),np.min(maxs))
print()

mins = []
maxs = []
for i,date_ in enumerate(dates_):
    date_ = datetime_to_astropy(date_)
    c_min,c_max = cor_range(date_,precession=False)
    mins += [c_min]
    maxs += [c_max]
print(mins)
print(maxs)
print("range (no precession):",np.max(mins),np.min(maxs))

delta: -0.08823603793466808
delta: -0.049049525816649106
delta: -0.008687038988408852
delta: 0.023747102212855635
delta: 0.06300948366701792
delta: 0.1021959957850369
delta: 0.14255848261327714
delta: 0.17503055848261329
delta: 0.21421707060063225
[3.4000000000000004, 3.6000000000000005, 3.2, 3.7000000000000006, 3.7000000000000006, 3.8000000000000007, 3.3000000000000003, 3.8000000000000007, 3.4000000000000004]
[4.700000000000001, 4.700000000000001, 4.300000000000001, 4.700000000000001, 4.700000000000001, 4.700000000000001, 4.500000000000002, 4.700000000000001, 4.700000000000001]
range: 3.8000000000000007 4.300000000000001

delta: 0
delta: 0
delta: 0
delta: 0
delta: 0
delta: 0
delta: 0
delta: 0
delta: 0
[3.5000000000000004, 3.6000000000000005, 3.2, 3.6000000000000005, 3.6000000000000005, 3.7000000000000006, 3.2, 3.6000000000000005, 3.2]
[4.700000000000001, 4.700000000000001, 4.300000000000001, 4.600000000000001, 4.700000000000001, 4.700000000000001, 4.400000000000001, 4.700000000000001,

In [ ]:
more_dates = [date(1980,5,19),date(1983,2,18),date(1985,7,22),date(1988,5,23)]

mins = []
maxs = []
for i,date_ in enumerate(more_dates):
    date_ = datetime_to_astropy(date_)
    c_min,c_max = cor_range(date_)
    mins += [c_min]
    maxs += [c_max]
print(mins)
print(maxs)
print("range:",np.max(mins),np.min(maxs))
print()

mins = []
maxs = []
for i,date_ in enumerate(more_dates):
    date_ = datetime_to_astropy(date_)
    c_min,c_max = cor_range(date_,precession=False)
    mins += [c_min]
    maxs += [c_max]
print(mins)
print(maxs)
print("range (no precession):",np.max(mins),np.min(maxs))


delta: -0.2718398314014753
delta: -0.2337154899894626
delta: -0.20014330874604846
delta: -0.16084299262381455
[3.2, 3.7000000000000006, 3.7000000000000006, 3.7000000000000006]
[3.900000000000001, 3.8000000000000007, 4.700000000000001, 4.700000000000001]
range: 3.7000000000000006 3.8000000000000007

delta: 0
delta: 0
delta: 0
delta: 0
[3.2, 4.000000000000001, 3.900000000000001, 3.900000000000001]
[4.200000000000001, 4.000000000000001, 4.700000000000001, 4.700000000000001]
range (no precession): 4.000000000000001 4.000000000000001


In [ ]:
more_dates = [date(1972,4,25),date(1974,8,20),date(1977,7,25)]

mins = []
maxs = []
for i,date_ in enumerate(more_dates):
    date_ = datetime_to_astropy(date_)
    c_min,c_max = cor_range(date_)
    mins += [c_min]
    maxs += [c_max]
print(mins)
print(maxs)
print("range:",np.max(mins),np.min(maxs))
print()

mins = []
maxs = []
for i,date_ in enumerate(more_dates):
    date_ = datetime_to_astropy(date_)
    c_min,c_max = cor_range(date_,precession=False)
    mins += [c_min]
    maxs += [c_max]
print(mins)
print(maxs)
print("range (no precession):",np.max(mins),np.min(maxs))

delta: -0.38359536354056906
delta: -0.3514646996838777
delta: -0.3108746048472076
[3.2, 3.2, 3.2]
[4.100000000000001, 4.500000000000002, 3.8000000000000007]
range: 3.2 3.8000000000000007

delta: 0
delta: 0
delta: 0
[3.2, 3.4000000000000004, 3.2]
[4.500000000000002, 4.700000000000001, 4.100000000000001]
range (no precession): 3.4000000000000004 4.100000000000001


In [ ]:
dates_ = [date(1972,4,25),date(1915,4,25),date(1934,4,25),date(1953,4,25)]
mins = []
maxs = []

for date_ in dates_:
    date_ = datetime_to_astropy(date_)
    sd,ed,masa,_ = find_masa_start_end_date(date_)

    c_min,c_max = cor_range(date_,accuracy=0.001,precession=False,start=3.8,end=4.3,step=0.02)
    mins += [c_min]
    maxs += [c_max]
    print(masa,sd,ed)
    print()

print(mins)
print(maxs)
print("range (no precession):",np.max(mins),np.min(maxs))

delta: 0
Vaisakha 1972-04-13 20:37:22.000 1972-05-13 04:00:32.000



/usr/local/lib/python3.7/dist-packages/erfa/core.py:155: ErfaWarning: ERFA function "dtf2d" yielded 1 of "dubious year (Note 6)"
  ErfaWarning)
/usr/local/lib/python3.7/dist-packages/erfa/core.py:155: ErfaWarning: ERFA function "d2dtf" yielded 1 of "dubious year (Note 5)"
  ErfaWarning)
/usr/local/lib/python3.7/dist-packages/erfa/core.py:155: ErfaWarning: ERFA function "utctai" yielded 1 of "dubious year (Note 3)"
  ErfaWarning)
/usr/local/lib/python3.7/dist-packages/erfa/core.py:155: ErfaWarning: ERFA function "taiutc" yielded 1 of "dubious year (Note 4)"
  ErfaWarning)


delta: 0
Vaisakha 1915-04-14 11:30:01.000 1915-05-14 03:28:11.000

delta: 0
Vaisakha 1934-04-13 23:56:12.000 1934-05-13 12:28:59.000

delta: 0
Vaisakha 1953-04-13 20:12:17.000 1953-05-13 05:03:17.000

[3.8, 3.8, 3.8, 3.8]
[4.28, 4.28, 4.28, 4.28]
range (no precession): 3.8 4.28


In [ ]:
dates_ = [date(1931,6,25),date(1950,6,25),date(1969,6,25),date(1920,7,25),date(1939,7,25),date(1958,7,25),date(1945,3,18),date(1945,3,18),date(1964,3,18)]
mins = []
maxs = []

for date_ in dates_:
    date_ = datetime_to_astropy(date_)
    sd,ed,masa,_ = find_masa_start_end_date(date_)

    c_min,c_max = cor_range(date_,accuracy=0.001,precession=False,start=3.8,end=4.3,step=0.02)
    mins += [c_min]
    maxs += [c_max]
    print(masa,sd,ed)
    print()

print(mins)
print(maxs)
print("range (no precession):",np.max(mins),np.min(maxs))

/usr/local/lib/python3.7/dist-packages/erfa/core.py:155: ErfaWarning: ERFA function "dtf2d" yielded 1 of "dubious year (Note 6)"
  ErfaWarning)
/usr/local/lib/python3.7/dist-packages/erfa/core.py:155: ErfaWarning: ERFA function "d2dtf" yielded 1 of "dubious year (Note 5)"
  ErfaWarning)
/usr/local/lib/python3.7/dist-packages/erfa/core.py:155: ErfaWarning: ERFA function "utctai" yielded 1 of "dubious year (Note 3)"
  ErfaWarning)
/usr/local/lib/python3.7/dist-packages/erfa/core.py:155: ErfaWarning: ERFA function "taiutc" yielded 1 of "dubious year (Note 4)"
  ErfaWarning)


delta: 0
Asadha 1931-06-16 03:06:15.000 1931-07-15 12:14:14.000

delta: 0
Asadha 1950-06-15 15:46:17.000 1950-07-15 04:53:15.000

delta: 0
Asadha 1969-06-14 23:07:12.000 1969-07-14 14:01:51.000

delta: 0
Sravana 1920-07-15 20:30:04.000 1920-08-14 03:39:45.000

delta: 0
Sravana 1939-07-16 21:07:22.000 1939-08-15 03:45:47.000

delta: 0
Sravana 1958-07-16 18:31:21.000 1958-08-15 03:30:57.000

delta: 0
Caitra 1945-03-14 03:44:20.000 1945-04-12 12:33:38.000

delta: 0
Caitra 1945-03-14 03:44:20.000 1945-04-12 12:33:38.000

delta: 0
Caitra 1964-03-14 02:22:37.000 1964-04-12 12:38:49.000

[3.8, 3.8, 3.8, 3.8, 3.8, 3.8, 3.8, 3.8, 3.8]
[4.28, 4.28, 4.220000000000001, 4.2, 4.28, 4.28, 4.08, 4.08, 4.140000000000001]
range (no precession): 3.8 4.08


----------------------------------------------------------------------------
### The date date(1983,2,18) seems to be constraining tremendously. Let's look at it in detail

In [ ]:

accuracy = 0.001
precession = False
date_ = datetime_to_astropy(date(1983,2,18))

st,et,masa,_ = find_masa_start_end_date(date_,accuracy=accuracy,cor=0.0) # st=start_time, et=end_time
_,ss = get_sun_moon_Ec(st) # ss,es = start and end sun positions respectively
_,es = get_sun_moon_Ec(et)
date_ = astropy_to_datetime(date_)
delta_ = (date_ - datetime(2000,1,1,0,0,0)).days/(26000*365)*360
if not precession: delta_=0
print("delta:",delta_)

cor_cor = []
for cor in np.arange(3.90,4.10,0.01):
    sr,er = find_rasi_Ec(ss.lon,cor=cor-delta_),find_rasi_Ec(es.lon,cor=cor-delta_) # sn,en = start and end rasi respectively.
    #sr = find_rasi_Ec(ss.lon,cor=cor)
    #er = find_rasi_Ec(es.lon,cor=cor)
    if sr==er:
      cor_cor += [cor]


print(cor_cor)
print(np.min(cor_cor),np.max(cor_cor))
print(masa)

delta: 0
[3.939999999999999, 3.949999999999999, 3.9599999999999986, 3.9699999999999984, 3.979999999999998, 3.989999999999998, 3.999999999999998, 4.009999999999998, 4.019999999999998, 4.029999999999998, 4.039999999999997, 4.049999999999997, 4.059999999999997, 4.069999999999997, 4.0799999999999965]
3.939999999999999 4.0799999999999965
Phalguna


----------------------------------------------------------------------------
### Best constraint so far: **[3.94, 4.08]**
### Actual Value of the Asvini starting angle = 19.879554934503027 + [3.94, 4.08] = **[23.82, 23.96]**
### Update: It is actually Lahiri ayanamsa, which is **23.8564** at J2000, i.e. cor = 19.879555 + 23.856389 = 3.977
----------------------------------------------------------------------------
Pro tip: This māsa is Phālguna. It has very tight constraints probably because the Earth moves very quickly through this Rāśī. Look for other Phalguna adhika māsa-s to get other strong constraints. Maybe the months near Phalguna, i.e. Magha, Caitra, Vaisakha, can also lead to constraints.

The conclusion about Lahiri ayanamsa is found by matching to www.drikpanchang.com

In [ ]:

cor = 23+51/60+23/3600-19.879554934503027
print(cor)

3.9768339543858637


### Calculating Lahiri Ayanamsa from definition, i.e. 180deg from Citra (Spica)

In [ ]:
# Coordinates of Spica in ICRS J2000 coordinates ra=13h25m11.579s, dec=−11°09′40.75″
coord_Spica_ICRS = SkyCoord(ra='13h25m11.579s', dec='−11d09m40.75s', frame='icrs', equinox=Time(2000, format='jyear')) 
coord_Spica_GTE = coord_Spica_ICRS.transform_to(GeocentricTrueEcliptic())
print("Citra coordinates:",coord_Spica_GTE)
ayanamsa = coord_Spica_GTE.lon.degree - 180
print("Citrapaksa ayanamsa:",ayanamsa)

Citra coordinates: <SkyCoord (GeocentricTrueEcliptic: equinox=2000.0, obstime=J2000.000): (lon, lat, distance) in (deg, deg, )
    (203.83614302, -2.05428596, 1.)>
Citrapaksa ayanamsa: 23.836143024753255


# Misc Stuff, Random Experiments

### predicting ksaya masa correctly. 

One before 1983 adhika (Magha is ksaya between Asvina and Phalguna),

another before 1964 adhika (Margasirsa is ksaya between Kartika and Caitra)

In [ ]:
# predicting ksaya masa correctly. 
# One before 1983 adhika (Magha is ksaya between Asvina and Phalguna),
# another before 1964 adhika (Margasirsa is ksaya between Kartika and Caitra)

d_s = date(1982,2,1)

dt = d_s
ms_l = []
i = 0
while dt<date(1984,2,1):
  if i%12==0: print(i/12)
  s,e,ms,_ = find_masa_start_end_date(t=datetime_to_astropy(dt),accuracy=0.01,cor=4.0)
  ms_l += [(ms,s)]
  dt = astropy_to_date(e) + timedelta(days=2)
  i += 1
print("Adhika Masa-s")
adhika = []
for i,(m,d) in enumerate(ms_l[0:-1]):
    (m2,_) = ms_l[i+1]
    if m==m2:
      print(m,d)
      adhika += [(m,d)]

print()
print("All Masa-s around the 2 adhikas")
for (m,d) in ms_l[7:-7]:
    print(m,d)

0.0
1.0
2.0
Adhika Masa-s
Asvina 1982-09-17 12:09:16.000
Phalguna 1983-02-13 00:31:30.000

All Masa-s around the 2 adhikas
Bhadrapada 1982-08-19 02:45:07.000
Asvina 1982-09-17 12:09:16.000
Asvina 1982-10-17 00:04:05.000
Kartika 1982-11-15 15:09:42.000
Mrgashirsha 1982-12-15 09:17:27.000
Pausa 1983-01-14 05:07:17.000
Phalguna 1983-02-13 00:31:30.000
Phalguna 1983-03-14 17:43:12.000
Caitra 1983-04-13 07:58:18.000
Vaisakha 1983-05-12 19:25:17.000
Jyestha 1983-06-11 04:37:55.000


In [ ]:
# predicting ksaya masa correctly. 
# One before 1983 adhika (Magha is ksaya between Asvina and Phalguna),
# another before 1964 adhika (Margasirsa is ksaya between Kartika and Caitra)

d_s = date(1963,2,1)

dt = d_s
ms_l = []
i = 0
while dt<date(1965,2,1):
  if i%12==0: print(i/12)
  s,e,ms,_ = find_masa_start_end_date(t=datetime_to_astropy(dt),accuracy=0.01,cor=4.0)
  ms_l += [(ms,s)]
  dt = astropy_to_date(e) + timedelta(days=2)
  i += 1
print("Adhika Masa-s")
adhika = []
for i,(m,d) in enumerate(ms_l[0:-1]):
    (m2,_) = ms_l[i+1]
    if m==m2:
      print(m,d)
      adhika += [(m,d)]

print()
print("All Masa-s around the 2 adhikas")
for (m,d) in ms_l[7:-7]:
    print(m,d)

0.0
1.0
2.0
Adhika Masa-s
Kartika 1963-10-17 12:41:59.000
Caitra 1964-03-14 02:13:08.000

All Masa-s around the 2 adhikas
Bhadrapada 1963-08-19 07:34:27.000
Asvina 1963-09-17 20:50:27.000
Kartika 1963-10-17 12:41:59.000
Kartika 1963-11-16 06:49:40.000
Pausa 1963-12-16 02:06:07.000
Magha 1964-01-14 20:43:07.000
Phalguna 1964-02-13 13:01:07.000
Caitra 1964-03-14 02:13:08.000
Caitra 1964-04-12 12:37:10.000
Vaisakha 1964-05-11 21:00:47.000
Jyestha 1964-06-10 04:22:04.000


### Predicting more Phalguna masa-s for tighter constraints

In [ ]:
from google.colab import files

uploaded = files.upload()

Saving listfile_2.txt to listfile_2 (1).txt
Saving listfile_3.txt to listfile_3 (1).txt
Saving listfile.txt to listfile (2).txt


In [ ]:
f1 = uploaded['listfile.txt'].decode("utf-8") 
f2 = uploaded['listfile_2.txt'].decode("utf-8") 
f3 = uploaded['listfile_3.txt'].decode("utf-8") 

In [ ]:
files = [f1,f2,f3]
new_files = []
for f in files:
  f = f.split('\n')
  f = [elem.split(',') for elem in f]
  temp = []
  for i,elem in enumerate(f):
      try:
        [ms,tm] = elem
        ms = ms[2:-1]
        tm = Time(tm[tm.rfind('=')+1:-2])
        temp += [(ms,tm)]
      except:
        print(i,elem)
  new_files += [temp]

/usr/local/lib/python3.7/dist-packages/erfa/core.py:155: ErfaWarning: ERFA function "dtf2d" yielded 1 of "dubious year (Note 6)"
  ErfaWarning)


613 ['']
965 ['']
521 ['']
